In [1]:
# Check if fidelity of the symmetric state with fermi sea state is preserved at 0.25

import qiskit
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit
import qiskit_aer 
from qiskit.quantum_info import state_fidelity
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.quantum_info.states.random import random_statevector
from qiskit.circuit.library import Initialize
from qiskit.visualization import plot_bloch_multivector
import numpy as np




In [5]:
coeff_dict  = {}

def array_k1(num_qubits):  #assuming num_qubits is even
    array_k = []
    m = num_qubits/2
    if (m)%2!=0: #replace with m-1 for previous results
        for j in range(-int((m)//2),int((m)//2) + 1): #replace with m-1 for previous results
            array_k.append(2*np.pi*j/num_qubits)
    else:
        for j in range(-int((m)//2),int((m)//2)):
            array_k.append(2*np.pi*j/num_qubits)
    return array_k


def recursive_nested(l,num_qubits,coeff_array,coeff = 1,bitstr=''):

    m = int(num_qubits/2) # taking always even number of qubits

    #coeff_dict_2 = {}
    if l==m-1: # m-1, but we start with 0 indexing
        coeff_copy = coeff  #to ensure multiplied coeffs in previous rounds is preserved and reused
        bitstr_copy = bitstr
        for i in range(num_qubits):
            if str(i) in bitstr:
                pass
            else:
                coeff = coeff*coeff_array[l,i]
                bitstr = bitstr + f'{i}'
                bitstr_sorted = sort_bitstr(bitstr) #to sort the string first
                perm = perm_str(bitstr,bitstr_sorted) # to compare hamming distance
                if bitstr_sorted in coeff_dict.keys():
                    coeff_dict[bitstr_sorted]+=coeff*perm
                else:
                    coeff_dict[bitstr_sorted] = coeff*perm
            bitstr = bitstr_copy
            coeff = coeff_copy
    if l!=m-1:
        coeff_copy = coeff  #to ensure multiplied coeffs in previous rounds is preserved and reused
        bitstr_copy = bitstr
        for i in range(num_qubits):
            if str(i) in bitstr:
                pass
            else:
                #print(coeff)
                coeff = coeff*coeff_array[l,i]
                bitstr += str(i)
                recursive_nested(l+1,num_qubits,coeff_array,coeff,bitstr)
            bitstr = bitstr_copy
            coeff = coeff_copy

    return coeff_dict

def sort_bitstr(bitstr):
    bit_array = []
    for i in bitstr:
        bit_array.append(i)
    #print(bit_array)
    bitstr_sorted = ''
    for i in range(len(bit_array)):
        bit_array[i] = int(bit_array[i])
    bit_array.sort()
    #print(bit_array)
    for i in bit_array:
        #print(i,str(i))
        bitstr_sorted += str(i)
        #print(bitstr_sorted)

    return bitstr_sorted

def hammingDist(str1, str2): 
    i = 0
    count = 0
  
    while(i < len(str1)): 
        if(str1[i] != str2[i]): 
            count += 1
        i += 1
    return count 

def perm_str(bitstr,bitstr_sorted):
    c = hammingDist(bitstr,bitstr_sorted)
    if c%2 == 0:
        if int(c/2)%2 == 0:
            return 1
        else:
            return -1
    else:
        if int(c/2)%2 == 0:
            return -1
        else:
            return 1



In [12]:
def fermi_state(num_qubits): #of the form num_qubits = 2*odd number


    coeff_dict.clear()
    m = int(num_qubits/2)
    coeff_array = []
    array_k = array_k1(num_qubits)
    for k in array_k:
        pos_list = []
        for x in range(num_qubits):
            pos_list.append(np.exp(-1j*k*x))
        coeff_array.append(pos_list)

    coeff_array = np.array(coeff_array)

    #print(coeff_array)

    coeff_dict_2 = recursive_nested(0,num_qubits,coeff_array)
    #print(coeff_dict_2)
    bitstr_dict = {}
    for bstr in coeff_dict_2.keys():
        vac_str = ''
        num_list = []
        for k in range(m):
            num_list.append(int(bstr[k]))

        for i in range(num_qubits):
            if i in num_list:
                vac_str += '1'
            else:
                vac_str += '0'
        bitstr_dict[vac_str] = coeff_dict[bstr]

    #print(bitstr_dict)
    

    fermi_state = Statevector([0]*(2**num_qubits))

    for bstr in bitstr_dict.keys():
        fermi_state += Statevector.from_label(bstr)*bitstr_dict[bstr]

    val_array = []
    for i in bitstr_dict.values():
        val_array.append(i)
    np_array = np.array(val_array)
    #print(np_array)
    #print(np.linalg.norm(np_array))
    
    fermi_state = fermi_state/np.linalg.norm(val_array)
    #print(fermi_state)
    fermi_state.is_valid()

    return fermi_state


print(fermi_state(6))

14.696938456699069
Statevector([ 0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
              0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
              0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
              0.00000000e+00+0.j        ,  7.55411086e-18+0.11785113j,
              0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
              0.00000000e+00+0.j        ,  7.55411086e-18+0.23570226j,
              0.00000000e+00+0.j        ,  0.00000000e+00+0.23570226j,
              7.55411086e-18+0.11785113j,  0.00000000e+00+0.j        ,
              0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
              0.00000000e+00+0.j        ,  7.55411086e-18+0.23570226j,
              0.00000000e+00+0.j        ,  0.00000000e+00+0.35355339j,
              0.00000000e+00+0.23570226j,  0.00000000e+00+0.j        ,
              0.00000000e+00+0.j        ,  0.00000000e+00+0.23570226j,
              0.00000000e+00+0.23570226j,  0.00000000e+00+

In [17]:
qubit_list = [6,10,14]

def create_sym_state(num_qubits):
    str1 = ''
    str2 = ''
    for i in range(num_qubits):
        if i%2 == 0:
            str1 += '1'
            str2 += '0'
        else:
            str1 += '0'
            str2 += '1'
    sym_state = (Statevector.from_label(str1) + Statevector.from_label(str2))/np.sqrt(2)
    return sym_state

def sym_state(qubit_list):
    state_list = []
    for num_qubits in qubit_list:
        state_list.append(create_sym_state(num_qubits))

    return state_list

def fermi_list(qubit_list):
    state_list = []
    for num_qubits in qubit_list:
        state_list.append(fermi_state(num_qubits))

    return state_list

def fid_sym_fermi (state_list, fermi_list):
    for i in range(len(state_list)):
        print("Fidelity of symmetric state for", 4*i + 2, "qubits with fermi sea state is", state_fidelity(state_list[i],fermi_list[i]))

In [19]:

fermi_list = fermi_list(qubit_list)




14.696938456699069
155.6559286276
888.0344275559656


In [20]:
state_list = sym_state(qubit_list)
fid_sym_fermi(state_list,fermi_list)

Fidelity of symmetric state for 0 qubits with fermi sea state is 0.25
Fidelity of symmetric state for 1 qubits with fermi sea state is 5.545826323345318e-05
Fidelity of symmetric state for 2 qubits with fermi sea state is 0.0


In [23]:
state_10 = (Statevector.from_label('1010101010') + Statevector.from_label("0101010101"))/np.sqrt(2)
state_fidelity(state_10,fermi_state(10))

155.6559286276


5.545826323345318e-05